In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from copy import deepcopy
from sklearn.metrics import adjusted_rand_score
from scipy.special import comb
from itertools import combinations
from scipy.stats import multivariate_normal

In [2]:
data=pd.read_csv('/Users/Supriya/Desktop/Data_Cortex_Nuclear.csv')

In [3]:
TL=data['class'].replace(data['class'].unique(),[0,1,2,3,4,5,6,7])
TL.unique()

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [4]:
data=data.iloc[:,1:78]
columns=data.columns.values

In [5]:
data.head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N
0,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,1.750936,...,0.188852,0.122652,NaN,0.106305,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652
1,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,1.596377,...,0.200404,0.116682,NaN,0.106592,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610
2,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,1.561316,...,0.193685,0.118508,NaN,0.108303,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427
3,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,1.595086,...,0.192112,0.132781,NaN,0.103184,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563
4,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,1.504230,...,0.205604,0.129954,NaN,0.104784,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730


In [6]:
col_null=data.columns[data.isnull().any()].values
col_null

array(['DYRK1A_N', 'ITSN1_N', 'BDNF_N', 'NR1_N', 'NR2A_N', 'pAKT_N',
       'pBRAF_N', 'pCAMKII_N', 'pCREB_N', 'pELK_N', 'pERK_N', 'pJNK_N',
       'PKCA_N', 'pMEK_N', 'pNR1_N', 'pNR2A_N', 'pNR2B_N', 'pPKCAB_N',
       'pRSK_N', 'AKT_N', 'BRAF_N', 'CAMKII_N', 'CREB_N', 'ELK_N',
       'ERK_N', 'GSK3B_N', 'JNK_N', 'MEK_N', 'TRKA_N', 'RSK_N', 'APP_N',
       'Bcatenin_N', 'SOD1_N', 'MTOR_N', 'P38_N', 'pMTOR_N', 'DSCR1_N',
       'AMPKA_N', 'NR2B_N', 'pNUMB_N', 'RAPTOR_N', 'TIAM1_N', 'pP70S6_N',
       'BAD_N', 'BCL2_N', 'pCFOS_N', 'H3AcK18_N', 'EGR1_N', 'H3MeK4_N'],
      dtype=object)

In [7]:
for i in col_null:
    data[i].fillna(data[i].mean(), inplace = True)

In [8]:
data.columns[data.isnull().any()].values

array([], dtype=object)

In [9]:
sc=StandardScaler()
X=pd.DataFrame(sc.fit_transform(data),columns=columns)

In [10]:
X.head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N
0,0.312711,0.517934,2.253669,1.497362,2.304365,-0.345019,-0.158601,-0.899902,0.604115,0.691475,...,-1.305403,-1.333224,-2.361356e-15,-1.066276,-0.987371,-0.285744,-1.011615,-1.416624,-1.607891,1.065901
1,0.356798,0.286501,1.880279,1.420009,1.976769,-0.518126,-0.334523,-0.963018,0.442658,0.359831,...,-0.906725,-1.558938,-2.361356e-15,-1.046167,-1.162114,-0.067645,-1.063458,-1.325218,-1.546844,1.280291
2,0.334966,0.450465,2.012928,1.124860,1.909149,-0.581298,-0.226891,-0.969835,0.543335,0.284600,...,-1.138611,-1.489897,-2.361356e-15,-0.926271,-1.079366,-0.155059,-1.065130,-1.373257,-1.623595,1.857038
3,0.065474,-0.000103,0.802123,0.489481,1.219245,-0.247424,-0.199461,-1.071196,-0.171236,0.357061,...,-1.192887,-0.950252,-2.361356e-15,-1.285019,-0.860192,-0.818989,-0.723367,-0.984718,-1.218364,1.144490
4,0.036682,0.001305,0.805692,0.197652,0.939202,-0.482759,-0.304516,-1.085341,-0.627671,0.162107,...,-0.727242,-1.057141,-2.361356e-15,-1.172895,-0.884881,-0.179502,-0.943378,-1.181435,-1.187585,1.583530


In [11]:
#Rand_index
def rand_index(labels_true, labels_pred):
    """given the true and predicted labels, 
    it will return the Rand Index.
    Args:
        labels_true: the real class labels 
        labels_pred:the predicted class labels
    Return:
        RAND_index
    """
    # create list of all combinations with the length of labels.
    my_pair = list(combinations(range(len(labels_true)), 2))

    def is_equal(x):
        return (x[0] == x[1])
    my_a = 0
    my_b = 0
    
    for i in range(len(my_pair)):
        if(is_equal((labels_true[my_pair[i][0]], labels_true[my_pair[i][1]])) == is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]]))
           and is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]])) == True):
            my_a += 1
        if(is_equal((labels_true[my_pair[i][0]], labels_true[my_pair[i][1]])) == is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]]))
           and is_equal((labels_pred[my_pair[i][0]], labels_pred[my_pair[i][1]])) == False):
            my_b += 1
            
    my_denom = comb(len(labels_true), 2)
    RAND_index = (my_a + my_b) / my_denom
    return RAND_index

In [12]:
def E_Step(X, mu, cov, alpha):
    N = X.shape[0]
    K = alpha.shape[0]
    h = np.mat(np.zeros((N, K)))
    prob = np.zeros((N, K))
    for k in range(K):
        prob[:, k] =multivariate_normal(mean=mu[k], cov=cov[k]).pdf(X) 
    prob = np.mat(prob)
    for k in range(K):
        h[:, k] = alpha[k] * prob[:, k]
    for i in range(N):
        h[i, :] /= np.sum(h[i, :])
    return h

In [13]:
def M_Step(X, h):
    N, D = X.shape
    K = h.shape[1]
    mu = np.zeros((K, D))
    cov = []
    alpha = np.zeros(K)

    for k in range(K):
        Nk = np.sum(h[:, k])
        # Update mu
        for d in range(D):
            mu[k, d] = np.sum(np.multiply(h[:, k], X[:, d])) / Nk
        # Update cov
        cov_k = np.mat(np.zeros((D, D)))
        for i in range(N):
            cov_k += h[i, k] * (X[i] - mu[k]).T * (X[i] - mu[k]) / Nk
        cov.append(cov_k)
        # Update alpha
        alpha[k] = Nk / N
    cov = np.array(cov)
    return mu, cov, alpha

In [14]:
def GMM_EM(X, K, times):
    N, D = X.shape
    mu = np.random.rand(K, D)
    cov = np.array([np.eye(D)] * K)
    alpha = np.array([1.0 / K] * K)
    for i in range(times):
        h = E_Step(X, mu, cov+1e-6*np.eye(D), alpha)
        mu, cov, alpha = M_Step(X, h)
    return mu, cov, alpha

In [15]:
matX=np.matrix(X, copy=True)
K=[2,3,4,5,6,7,8,9,10]
iterations=100
for k in K:
    mu, cov, alpha = GMM_EM(matX, k, iterations)
    h = E_Step(matX, mu, cov+1e-6*np.eye(77), alpha)
    cluster=h.argmax(axis=1).flatten().tolist()[0]
    print('For K= %d\nRand_Index=%f\nAdjusted_Rand_Index=%f\n'%(k,rand_index(TL,cluster),adjusted_rand_score(TL,cluster)))

For K= 2
Rand_Index=0.521927
Adjusted_Rand_Index=0.067668

For K= 3
Rand_Index=0.638276
Adjusted_Rand_Index=0.091194

For K= 4
Rand_Index=0.698116
Adjusted_Rand_Index=0.066512

For K= 5
Rand_Index=0.743674
Adjusted_Rand_Index=0.094745

For K= 6
Rand_Index=0.753973
Adjusted_Rand_Index=0.058239

For K= 7
Rand_Index=0.770527
Adjusted_Rand_Index=0.084048

For K= 8
Rand_Index=0.795819
Adjusted_Rand_Index=0.100749

For K= 9
Rand_Index=0.782219
Adjusted_Rand_Index=0.071064

For K= 10
Rand_Index=0.804246
Adjusted_Rand_Index=0.066359

